In [ ]:
# # Use GFPGAN folder from drive
# from google.colab import drive
# drive.mount('/content/drive')
# drive_path = '/content/drive/MyDrive/GFPColab';
# # %mkdir -p $drive_path
# # %cp -r ../GFPGAN $drive_path
# %cd $drive_path/GFPGAN

# # %env BASICSR_JIT=True
# %env BASICSR_EXT=True

# # Clone GFPGAN and enter the GFPGAN folder
# # %cd /content
# # !rm -rf GFPGAN
# # !git clone https://github.com/TencentARC/GFPGAN.git
# # %cd GFPGAN

# # Set up the environment
# # Install basicsr - https://github.com/xinntao/BasicSR
# # We use BasicSR for both training and inference
# !pip install basicsr
# # Install facexlib - https://github.com/xinntao/facexlib
# # We use face detection and face restoration helper in the facexlib package
# !pip install facexlib
# # Install other depencencies
# !pip install -r requirements.txt
# !python setup.py develop
# !pip install realesrgan  # used for enhancing the background (non-face) regions
# # Download the pre-trained model
# # !wget https://github.com/TencentARC/GFPGAN/releases/download/v0.2.0/GFPGANCleanv1-NoCE-C2.pth -P experiments/pretrained_models

In [1]:
%cd repo
!pwd

/media/server_admin/890419dd-5419-4fdf-ae01-37ee1e1a2185/raj_gfpgan/repo
/media/server_admin/890419dd-5419-4fdf-ae01-37ee1e1a2185/raj_gfpgan/repo


# Workspace

In [2]:
# !BASICSR_JIT=True BASICSR_EXT=True pip install basicsr

In [3]:
# import torch
# from gfpgan.archs.gfpganv1_arch import GFPGANv1

# gfpgan = GFPGANv1(
#                 out_size=512,
#                 num_style_feat=512,
#                 channel_multiplier=1,
#                 decoder_load_path=None,
#                 fix_decoder=True,
#                 num_mlp=8,
#                 input_is_latent=True,
#                 different_w=True,
#                 narrow=1,
#                 sft_half=True)
# model_path = 'experiments/pretrained_models/GFPGANv1.pth'
# loadnet = torch.load(model_path)
# if 'params_ema' in loadnet:
#     keyname = 'params_ema'
# else:
#     keyname = 'params'
# gfpgan.load_state_dict(loadnet[keyname], strict=True)

# gfpgan.eval()
# print()

In [4]:
from gfpgan.my_options_parser import my_parse_options

args = {}
args['opt'] = 'options/my_train_gfpgan_v1.yml'
args['launcher'] = 'none'
args['auto_resume'] = False
args['debug'] = False
args['local_rank'] = 0
args['force_yml'] = None
root_path = '/media/server_admin/890419dd-5419-4fdf-ae01-37ee1e1a2185/raj_gfpgan/repo'
opt_file = 'options/my_train_gfpgan_v1.yml'
opt, args = my_parse_options(root_path, args)

print(opt)
opt['root_path'] = root_path

/media/server_admin/890419dd-5419-4fdf-ae01-37ee1e1a2185/raj_gfpgan/venv2/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/media/server_admin/890419dd-5419-4fdf-ae01-37ee1e1a2185/raj_gfpgan/repo {'opt': 'options/my_train_gfpgan_v1.yml', 'launcher': 'none', 'auto_resume': False, 'debug': False, 'local_rank': 0, 'force_yml': None}
Disable distributed.
OrderedDict([('name', 'train_GFPGANv1_512'), ('model_type', 'GFPGANModel'), ('num_gpu', 1), ('manual_seed', 0), ('datasets', OrderedDict([('train', OrderedDict([('name', 'FFHQ'), ('type', 'FFHQDegradationDataset'), ('dataroot_gt', 'datasets/images_gt'), ('io_backend', OrderedDict([('type', 'disk')])), ('use_hflip', True), ('mean', [0.5, 0.5, 0.5]), ('std', [0.5, 0.5, 0.5]), ('out_size', 512), ('blur_kernel_size', 41), ('kernel_list', ['iso', 'aniso']), ('kernel_prob', [0.5, 0.5]), ('blur_sigma', [0.1, 10]), ('downsample_range', [0.8, 8]), ('noise_range', [0, 20]), ('jpeg_range', [60, 100]), ('color_jitter_prob', 0.3), ('color_jitter_shift', 20), ('color_jitter_pt_prob', 0.3), ('gray_prob', 0.01), ('crop_components', True), ('component_path', 'experiments/pretr

In [5]:
import torch
import logging
import time
import datetime
from os import path as osp
import numpy as np
import random

from basicsr.train import load_resume_state, init_tb_loggers
# from basicsr.train import create_train_val_dataloader


from basicsr.data import build_dataloader, build_dataset
from basicsr.data.prefetch_dataloader import CPUPrefetcher, CUDAPrefetcher
from basicsr.models import build_model
from basicsr.utils import (AvgTimer, MessageLogger, check_resume, get_env_info, get_root_logger, get_time_str,
                           init_tb_logger, init_wandb_logger, make_exp_dirs, mkdir_and_rename, scandir)
from basicsr.utils.options import copy_opt_file, dict2str, parse_options
from functools import partial

from basicsr.utils.dist_util import get_dist_info

In [6]:
def worker_init_fn(worker_id, num_workers, rank, seed):
    # Set the worker seed to num_workers * rank + worker_id + seed
    worker_seed = num_workers * rank + worker_id + seed
    np.random.seed(worker_seed)
    random.seed(worker_seed)

def build_dataloader(dataset, dataset_opt, num_gpu=1, dist=False, sampler=None, seed=None, batch_sampler=None):
    """Build dataloader.

    Args:
        dataset (torch.utils.data.Dataset): Dataset.
        dataset_opt (dict): Dataset options. It contains the following keys:
            phase (str): 'train' or 'val'.
            num_worker_per_gpu (int): Number of workers for each GPU.
            batch_size_per_gpu (int): Training batch size for each GPU.
        num_gpu (int): Number of GPUs. Used only in the train phase.
            Default: 1.
        dist (bool): Whether in distributed training. Used only in the train
            phase. Default: False.
        sampler (torch.utils.data.sampler): Data sampler. Default: None.
        seed (int | None): Seed. Default: None
    """
    phase = dataset_opt['phase']
    rank, _ = get_dist_info()
    if phase == 'train':
        if dist:  # distributed training
            batch_size = dataset_opt['batch_size_per_gpu']
            num_workers = dataset_opt['num_worker_per_gpu']
        else:  # non-distributed training
            multiplier = 1 if num_gpu == 0 else num_gpu
            batch_size = dataset_opt['batch_size_per_gpu'] * multiplier
            num_workers = dataset_opt['num_worker_per_gpu'] * multiplier
        dataloader_args = dict(
            dataset=dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=num_workers,
            sampler=sampler,
            drop_last=True)
        if batch_sampler is not None:
          dataloader_args = dict(
            dataset=dataset,
            num_workers=num_workers,
            batch_sampler=batch_sampler)
        # if sampler is None:
        #     dataloader_args['shuffle'] = True
        dataloader_args['worker_init_fn'] = partial(
            worker_init_fn, num_workers=num_workers, rank=rank, seed=seed) if seed is not None else None
    elif phase in ['val', 'test']:  # validation
        dataloader_args = dict(dataset=dataset, batch_size=1, shuffle=False, num_workers=0)
    else:
        raise ValueError(f'Wrong dataset phase: {phase}. ' "Supported ones are 'train', 'val' and 'test'.")

    dataloader_args['pin_memory'] = dataset_opt.get('pin_memory', False)
    dataloader_args['persistent_workers'] = dataset_opt.get('persistent_workers', False)

    prefetch_mode = dataset_opt.get('prefetch_mode')
    if prefetch_mode == 'cpu':  # CPUPrefetcher
        num_prefetch_queue = dataset_opt.get('num_prefetch_queue', 1)
        logger = get_root_logger()
        logger.info(f'Use {prefetch_mode} prefetch dataloader: num_prefetch_queue = {num_prefetch_queue}')
        return PrefetchDataLoader(num_prefetch_queue=num_prefetch_queue, **dataloader_args)
    else:
        # prefetch_mode=None: Normal dataloader
        # prefetch_mode='cuda': dataloader for CUDAPrefetcher
        return torch.utils.data.DataLoader(**dataloader_args)

In [7]:
import math
import torch
from torch.utils.data.sampler import Sampler


class EnlargedSampler(Sampler):
    """Sampler that restricts data loading to a subset of the dataset.

    Modified from torch.utils.data.distributed.DistributedSampler
    Support enlarging the dataset for iteration-based training, for saving
    time when restart the dataloader after each epoch

    Args:
        dataset (torch.utils.data.Dataset): Dataset used for sampling.
        num_replicas (int | None): Number of processes participating in
            the training. It is usually the world_size.
        rank (int | None): Rank of the current process within num_replicas.
        ratio (int): Enlarging ratio. Default: 1.
    """

    def __init__(self, dataset, num_replicas, rank, ratio=1, batch_size=3):
        self.dataset = dataset
        self.num_replicas = num_replicas
        self.rank = rank
        self.epoch = 0
        self.batch_size = batch_size
        self.num_samples = math.ceil(len(self.dataset) * ratio / self.num_replicas)
        self.total_size = self.num_samples * self.num_replicas
        self.batch_indices = []
        for batch in self.dataset.batch_paths:
          batch_index = []
          for id in batch:
            if len(batch_index) >= self.batch_size:
              self.batch_indices.append(batch_index)
              batch_index = []
            batch_index.append(id)
        print('======================= BATCH SIZE')
        print(self.batch_size)
#         print(self.batch_indices)
        

    def __iter__(self):
        # print(self.dataset.batch_paths)
        return iter(self.batch_indices)

    def __len__(self):
        return self.num_samples

    def set_epoch(self, epoch):
        self.epoch = epoch

In [8]:
# code for the loader

def create_train_val_dataloader(opt, logger):
    # create train and val dataloaders
    train_loader, val_loaders = None, []
    for phase, dataset_opt in opt['datasets'].items():
        if phase == 'train':
            dataset_enlarge_ratio = dataset_opt.get('dataset_enlarge_ratio', 1)
            train_set = build_dataset(dataset_opt)
            train_sampler = EnlargedSampler(train_set, opt['world_size'], opt['rank'], dataset_enlarge_ratio, batch_size=dataset_opt['batch_size_per_gpu'])
            train_loader = build_dataloader(
                train_set,
                dataset_opt,
                num_gpu=opt['num_gpu'],
                dist=opt['dist'],
                batch_sampler=train_sampler,
                seed=opt['manual_seed'])

            num_iter_per_epoch = math.ceil(
                len(train_set) * dataset_enlarge_ratio / (dataset_opt['batch_size_per_gpu'] * opt['world_size']))
            total_iters = int(opt['train']['total_iter'])
            total_epochs = math.ceil(total_iters / (num_iter_per_epoch))
            logger.info('Training statistics:'
                        f'\n\tNumber of train images: {len(train_set)}'
                        f'\n\tDataset enlarge ratio: {dataset_enlarge_ratio}'
                        f'\n\tBatch size per gpu: {dataset_opt["batch_size_per_gpu"]}'
                        f'\n\tWorld size (gpu number): {opt["world_size"]}'
                        f'\n\tRequire iter number per epoch: {num_iter_per_epoch}'
                        f'\n\tTotal epochs: {total_epochs}; iters: {total_iters}.')
        elif phase.split('_')[0] == 'val':
            val_set = build_dataset(dataset_opt)
            val_loader = build_dataloader(
                val_set, dataset_opt, num_gpu=opt['num_gpu'], dist=opt['dist'], sampler=None, seed=opt['manual_seed'])
            logger.info(f'Number of val images/folders in {dataset_opt["name"]}: {len(val_set)}')
            val_loaders.append(val_loader)
        else:
            raise ValueError(f'Dataset phase {phase} is not recognized.')

    return train_loader, train_sampler, val_loaders, total_epochs, total_iters

In [9]:
root_path = '/media/server_admin/890419dd-5419-4fdf-ae01-37ee1e1a2185/raj_gfpgan/repo'
torch.backends.cudnn.benchmark = True

resume_state = load_resume_state(opt)
# mkdir for experiments and logger
if resume_state is None:
  make_exp_dirs(opt)
  if opt['logger'].get('use_tb_logger') and 'debug' not in opt['name'] and opt['rank'] == 0:
    mkdir_and_rename(osp.join(opt['root_path'], 'tb_logger', opt['name']))


log_file = osp.join(opt['path']['log'], f"train_{opt['name']}_{get_time_str()}.log")
logger = get_root_logger(logger_name='basicsr', log_level=logging.INFO, log_file=log_file)
logger.info(get_env_info())
logger.info(dict2str(opt))
# initialize wandb and tb loggers
tb_logger = init_tb_loggers(opt)

# create train and validation dataloaders
result = create_train_val_dataloader(opt, logger)
train_loader, train_sampler, val_loaders, total_epochs, total_iters = result

model = build_model(opt)

Path already exists. Rename it to /media/server_admin/890419dd-5419-4fdf-ae01-37ee1e1a2185/raj_gfpgan/repo/tb_logger/train_GFPGANv1_512_archived_20220502_150623


2022-05-02 15:06:23,036 INFO: 
                ____                _       _____  ____
               / __ ) ____ _ _____ (_)_____/ ___/ / __ \
              / __  |/ __ `// ___// // ___/\__ \ / /_/ /
             / /_/ // /_/ /(__  )/ // /__ ___/ // _, _/
            /_____/ \__,_//____//_/ \___//____//_/ |_|
     ______                   __   __                 __      __
    / ____/____   ____   ____/ /  / /   __  __ _____ / /__   / /
   / / __ / __ \ / __ \ / __  /  / /   / / / // ___// //_/  / /
  / /_/ // /_/ // /_/ // /_/ /  / /___/ /_/ // /__ / /<    /_/
  \____/ \____/ \____/ \____/  /_____/\____/ \___//_/|_|  (_)
    
Version Information: 
	BasicSR: 1.3.5
	PyTorch: 1.11.0+cu102
	TorchVision: 0.12.0+cu102
2022-05-02 15:06:23,037 INFO: 
  name: train_GFPGANv1_512
  model_type: GFPGANModel
  num_gpu: 1
  manual_seed: 0
  datasets:[
    train:[
      name: FFHQ
      type: FFHQDegradationDataset
      dataroot_gt: datasets/images_gt
      io_backend:[
        type: disk
      ]
 

======================= BATCH SIZE
8
###### out_size512


2022-05-02 15:06:30,452 INFO: Network [GFPGANv1] is created.
2022-05-02 15:06:30,554 INFO: Network: GFPGANv1, with parameters: 78,308,493
2022-05-02 15:06:30,554 INFO: GFPGANv1(
  (conv_body_first): ConvLayer(
    (0): EqualConv2d(in_channels=3, out_channels=16, kernel_size=1, stride=1, padding=0, bias=False)
    (1): FusedLeakyReLU()
  )
  (conv_body_down): ModuleList(
    (0): ResBlock(
      (conv1): ConvLayer(
        (0): EqualConv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1, bias=False)
        (1): FusedLeakyReLU()
      )
      (conv2): ConvLayer(
        (0): UpFirDnSmooth(upsample_factor=1, downsample_factor=2)
        (1): EqualConv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=0, bias=False)
        (2): FusedLeakyReLU()
      )
      (skip): ConvLayer(
        (0): UpFirDnSmooth(upsample_factor=1, downsample_factor=2)
        (1): EqualConv2d(in_channels=16, out_channels=32, kernel_size=1, stride=2, padding=0, bias=False)
   

2022-05-02 15:06:30,894 INFO: Loading GFPGANv1 model from experiments/pretrained_models/GFPGANv1.pth, with param key: [params_ema].
2022-05-02 15:06:30,962 WARNING: Current net - loaded net:
2022-05-02 15:06:30,963 WARNING:   lstm_fusion_param
2022-05-02 15:06:30,963 WARNING:   style_lstm.bias_hh_l0
2022-05-02 15:06:30,964 WARNING:   style_lstm.bias_ih_l0
2022-05-02 15:06:30,964 WARNING:   style_lstm.weight_hh_l0
2022-05-02 15:06:30,964 WARNING:   style_lstm.weight_ih_l0
2022-05-02 15:06:30,964 WARNING: Loaded net - current net:
2022-05-02 15:06:31,189 INFO: Network [StyleGAN2Discriminator] is created.
2022-05-02 15:06:31,216 INFO: Network: StyleGAN2Discriminator, with parameters: 24,030,753
2022-05-02 15:06:31,217 INFO: StyleGAN2Discriminator(
  (conv_body): Sequential(
    (0): ConvLayer(
      (0): EqualConv2d(in_channels=3, out_channels=32, kernel_size=1, stride=1, padding=0, bias=False)
      (1): FusedLeakyReLU()
    )
    (1): ResBlock(
      (conv1): ConvLayer(
        (0): Equ

###### out_size512


2022-05-02 15:06:31,897 INFO: Network [GFPGANv1] is created.
2022-05-02 15:06:32,309 INFO: Loading GFPGANv1 model from experiments/pretrained_models/GFPGANv1.pth, with param key: [params_ema].
2022-05-02 15:06:32,375 WARNING: Current net - loaded net:
2022-05-02 15:06:32,375 WARNING:   lstm_fusion_param
2022-05-02 15:06:32,376 WARNING:   style_lstm.bias_hh_l0
2022-05-02 15:06:32,376 WARNING:   style_lstm.bias_ih_l0
2022-05-02 15:06:32,376 WARNING:   style_lstm.weight_hh_l0
2022-05-02 15:06:32,376 WARNING:   style_lstm.weight_ih_l0
2022-05-02 15:06:32,377 WARNING: Loaded net - current net:
2022-05-02 15:06:32,487 INFO: Network [FacialComponentDiscriminator] is created.
2022-05-02 15:06:32,490 INFO: Network: FacialComponentDiscriminator, with parameters: 1,110,785
2022-05-02 15:06:32,490 INFO: FacialComponentDiscriminator(
  (conv1): ConvLayer(
    (0): EqualConv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False)
    (1): FusedLeakyReLU()
  )
  (conv2): Con

2022-05-02 15:06:34,275 INFO: Loading ResNetArcFace model from experiments/pretrained_models/arcface_resnet18.pth, with param key: [None].
2022-05-02 15:06:34,301 WARNING: Current net - loaded net:
2022-05-02 15:06:34,301 WARNING:   bn1.num_batches_tracked
2022-05-02 15:06:34,302 WARNING:   bn4.num_batches_tracked
2022-05-02 15:06:34,302 WARNING:   bn5.num_batches_tracked
2022-05-02 15:06:34,302 WARNING:   layer1.0.bn0.num_batches_tracked
2022-05-02 15:06:34,303 WARNING:   layer1.0.bn1.num_batches_tracked
2022-05-02 15:06:34,303 WARNING:   layer1.0.bn2.num_batches_tracked
2022-05-02 15:06:34,303 WARNING:   layer1.1.bn0.num_batches_tracked
2022-05-02 15:06:34,303 WARNING:   layer1.1.bn1.num_batches_tracked
2022-05-02 15:06:34,304 WARNING:   layer1.1.bn2.num_batches_tracked
2022-05-02 15:06:34,304 WARNING:   layer2.0.bn0.num_batches_tracked
2022-05-02 15:06:34,304 WARNING:   layer2.0.bn1.num_batches_tracked
2022-05-02 15:06:34,304 WARNING:   layer2.0.bn2.num_batches_tracked
2022-05-02 15

In [10]:
print(total_iters)

50000


In [ ]:
if resume_state:  # resume training
  model.resume_training(resume_state)  # handle optimizers and schedulers
  logger.info(f"Resuming training from epoch: {resume_state['epoch']}, " f"iter: {resume_state['iter']}.")
  start_epoch = resume_state['epoch']
  current_iter = resume_state['iter']
else:
  start_epoch = 0
  current_iter = 0

msg_logger = MessageLogger(opt, current_iter, tb_logger)

prefetch_mode = opt['datasets']['train'].get('prefetch_mode')
if prefetch_mode is None or prefetch_mode == 'cpu':
  prefetcher = CPUPrefetcher(train_loader)
elif prefetch_mode == 'cuda':
  prefetcher = CUDAPrefetcher(train_loader, opt)
  logger.info(f'Use {prefetch_mode} prefetch dataloader')
  if opt['datasets']['train'].get('pin_memory') is not True:
      raise ValueError('Please set pin_memory=True for CUDAPrefetcher.')
else:
  raise ValueError(f'Wrong prefetch_mode {prefetch_mode}')

logger.info(f'Start training from epoch: {start_epoch}, iter: {current_iter}')
data_timer, iter_timer = AvgTimer(), AvgTimer()
start_time = time.time()


for epoch in range(start_epoch, total_epochs + 1):
  train_sampler.set_epoch(epoch)
  prefetcher.reset()
  train_data = prefetcher.next()

  while train_data is not None:
#     print('iter')
    data_timer.record()

    current_iter += 1
    if current_iter > total_iters:
      break
    # update learning rate
    model.update_learning_rate(current_iter, warmup_iter=opt['train'].get('warmup_iter', -1))
    # training
    model.feed_data(train_data)
    model.optimize_parameters(current_iter)
    iter_timer.record()
    if current_iter == 1:
      # reset start time in msg_logger for more accurate eta_time
      # not work in resume mode
      msg_logger.reset_start_time()
    # log
    if current_iter % opt['logger']['print_freq'] == 0:
      log_vars = {'epoch': epoch, 'iter': current_iter}
      log_vars.update({'lrs': model.get_current_learning_rate()})
      log_vars.update({'time': iter_timer.get_avg_time(), 'data_time': data_timer.get_avg_time()})
      log_vars.update(model.get_current_log())
      msg_logger(log_vars)

    # save models and training states
    if current_iter % opt['logger']['save_checkpoint_freq'] == 0:
      logger.info('Saving models and training states.')
      model.save(epoch, current_iter)

    # validation
    if opt.get('val') is not None and (current_iter % opt['val']['val_freq'] == 0):
      if len(val_loaders) > 1:
        logger.warning('Multiple validation datasets are *only* supported by SRModel.')
      for val_loader in val_loaders:
        model.validation(val_loader, current_iter, tb_logger, opt['val']['save_img'])

    data_timer.start()
    iter_timer.start()
    train_data = prefetcher.next()
    # end of iter

# end of epoch
e
consumed_time = str(datetime.timedelta(seconds=int(time.time() - start_time)))
logger.info(f'End of training. Time consumed: {consumed_time}')
logger.info('Save the latest model.')
model.save(epoch=-1, current_iter=-1)  # -1 stands for the latest

if tb_logger:
  tb_logger.close()

2022-05-02 15:06:34,600 INFO: Start training from epoch: 0, iter: 0


In [ ]:
!pwd
model

In [ ]:
# from gfpgan.data.ffhq_degradation_dataset import FFHQDegradationDataset

# option_train_dataset = options['dataset']['train']

# dataset = FFHQDegradationDataset(option_train_dataset)
# dataset_enlarge_ratio = option_train_dataset['dataset_enlarge_ratio']

# train_sampler = EnlargedSampler(train_set, train_data_opts['world_size'], train_data_opts['rank'], dataset_enlarge_ratio)


In [ ]:
# import os
# import cv2

# for img_path in os.listdir('./ffhq_small'):
#   print(img_path)
#   img = cv2.imread('./ffhq_small/' + img_path)
#   img_512 = cv2.resize(img, (512, 512))
#   cv2.imwrite('./ffhq_small_512/' + img_path, img_512)

## Test


In [ ]:
# upload your own images
import os
from google.colab import files
import shutil

upload_folder = 'inputs/upload'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
os.mkdir(upload_folder)

# upload images
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

In [ ]:
# !pwd
# %cd $drive_path
# %pwd

## 3. Inference

In [ ]:
# Now we use the GFPGAN to restore the above low-quality images
# We use [Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN) for enhancing the background (non-face) regions
!rm -rf results
!python inference_gfpgan.py --upscale 2 --test_path inputs/upload --save_root results --model_path experiments/pretrained_models/GFPGANCleanv1-NoCE-C2.pth --bg_upsampler none --only_center_face

!ls results/cmp

## 4. Visualize

In [ ]:
# We first visualize the cropped faces
# The left are the inputs images; the right are the results of GFPGAN

import cv2
import matplotlib.pyplot as plt
def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1) 
  plt.title('Input image', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('GFPGAN output', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

# display each image in the upload folder
import os
import glob

input_folder = 'results/cropped_faces'
result_folder = 'results/restored_faces'
input_list = sorted(glob.glob(os.path.join(input_folder, '*')))
output_list = sorted(glob.glob(os.path.join(result_folder, '*')))
for input_path, output_path in zip(input_list, output_list):
  img_input = imread(input_path)
  img_output = imread(output_path)
  display(img_input, img_output)


In [ ]:
# We then visualize the whole image
# The left are the inputs images; the right are the results of GFPGAN

import cv2
import matplotlib.pyplot as plt
def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1) 
  plt.title('Input image', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('GFPGAN output', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

# display each image in the upload folder
import os
import glob

input_folder = 'inputs/upload'
result_folder = 'results/restored_imgs'
input_list = sorted(glob.glob(os.path.join(input_folder, '*')))
output_list = sorted(glob.glob(os.path.join(result_folder, '*')))
for input_path, output_path in zip(input_list, output_list):
  img_input = imread(input_path)
  img_output = imread(output_path)
  display(img_input, img_output)

## 5. Download results

In [ ]:
# download the result
!ls results
print('Download results')
os.system('zip -r download.zip results')
files.download("download.zip")